In [1]:
# reload magics
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datasets
from project_modules.io import load_dataset_to_df
from project_modules.classifcation import classify_MP,getXY, boruta_fs
from project_modules.utils import MPutils
from sklearn.ensemble import RandomForestClassifier
import multiprocessing
from sklearn.model_selection import cross_val_score
import cupy as cp
from datetime import datetime

from tqdm.notebook import tqdm
from copy import deepcopy

# from project_modules.utils import get_logger
# logger = get_logger("log-data-combine-split.log")
# # read the parameter file

# from project_modules.utils import read_parameters
# parms = read_parameters("/Users/david/projects/lc-project-data/project.yaml")

pd.options.display.max_columns = None
pd.options.display.max_rows = 50

In [2]:
lScorersBinary = [
    "accuracy",
    "balanced_accuracy",
    "roc_auc",
    "f1",
    "recall",
    "sensitivity",
    "specificity",
    "precision",
    # "average_precision",
    "NPV",
    "PPV",
    # "neg_mean_squared_error",
]
lResCol = [
    "Title",
    "cv",
    "param_clf",
    "param_clf__max_depth",
    "param_clf__n_estimators",
    "param_clf__random_state",
    # "param_clf__max_iter",
    "mean_test_accuracy",
    "mean_test_balanced_accuracy",
    "mean_test_roc_auc",
    "mean_test_f1",
    "mean_test_recall",
    "mean_test_sensitivity",
    "mean_test_specificity",
    "mean_test_precision",
    "mean_test_NPV",
    "mean_test_PPV",
]

In [3]:
site_name_dict = {1: "LDN", 2: "MTL", 3: "SAN", 4: "RIO", 5: "LUS", 6: "CA", 7: "NA"}
status_name_dict = {0: "HC", 1: "LC_NEG", 2: "LC_POS"}

# Combine Results

In [4]:
import glob

path = "OUTPUT/MP/05-classifiers/DataV3"
files = glob.glob(path + '/*/CA*.csv')
# files = files+glob.glob(path + "/*/FS*.csv")
files = files + glob.glob(path + "/*/Val*.csv")
files = files+glob.glob(path+'/*/CLFRun*.csv')
files = [x for x in files if 'selcol' not in x.lower()]

print(files)
print(len(files))

['OUTPUT/MP/05-classifiers/DataV3/dfcmpltPreLC_SITE5/CA_dfcmpltPreLC_SITE5_Holdout.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmpltPreLC_SITE5/CA_dfcmpltPreLC_SITE5_Full.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmplt_SITE2/CA_dfcmplt_SITE2_Holdout.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmplt_SITE2/CA_dfcmplt_SITE2_Full.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmpltPreLC3_SITE3/CA_dfcmpltPreLC3_SITE3_Full.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmpltPreLC3_SITE3/CA_dfcmpltPreLC3_SITE3_Holdout.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmpltPreLC4_SITE4/CA_dfcmpltPreLC4_SITE4_Full.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmpltPreLC4_SITE4/CA_dfcmpltPreLC4_SITE4_Holdout.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmpltPreLC3_SITE4/CA_dfcmpltPreLC3_SITE4_Holdout.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmpltPreLC3_SITE4/CA_dfcmpltPreLC3_SITE4_Full.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmpltPreLC3_SITE5/CA_dfcmpltPreLC3_SITE5_Holdout.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmpltPreLC3_SITE5

In [5]:
ldf = []
for i in files:
    ldf.append(pd.read_csv(i))

In [6]:
dfRes = pd.concat(ldf)

In [8]:
def changeSite(x):
    if pd.isnull(x):
        return np.nan
    if 'SITE7' in x:
        return 'NorthAmerica'
    if 'SITE6' in x:
        return 'Canada'
    if 'SITE5' in x:
        return 'LUS'
    if 'SITE4' in x:
        return 'RIO'
    if 'SITE3' in x:
        return 'SAN'
    if 'SITE2' in x:
        return 'MTL'
    if 'SITE1' in x:
        return 'LDN'
    return "Global"

In [9]:
dfRes

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy,split0_test_balanced_accuracy,split1_test_balanced_accuracy,split2_test_balanced_accuracy,split3_test_balanced_accuracy,split4_test_balanced_accuracy,mean_test_balanced_accuracy,std_test_balanced_accuracy,rank_test_balanced_accuracy,split0_test_roc_auc,split1_test_roc_auc,split2_test_roc_auc,split3_test_roc_auc,split4_test_roc_auc,mean_test_roc_auc,std_test_roc_auc,rank_test_roc_auc,split0_test_f1,split1_test_f1,split2_test_f1,split3_test_f1,split4_test_f1,mean_test_f1,std_test_f1,rank_test_f1,split0_test_recall,split1_test_recall,split2_test_recall,split3_test_recall,split4_test_recall,mean_test_recall,std_test_recall,rank_test_recall,split0_test_sensitivity,split1_test_sensitivity,split2_test_sensitivity,split3_test_sensitivity,split4_test_sensitivity,mean_test_sensitivity,std_test_sensitivity,rank_test_sensitivity,split0_test_specificity,split1_test_specificity,split2_test_specificity,split3_test_specificity,split4_test_specificity,mean_test_specificity,std_test_specificity,rank_test_specificity,split0_test_precision,split1_test_precision,split2_test_precision,split3_test_precision,split4_test_precision,mean_test_precision,std_test_precision,rank_test_precision,split0_test_NPV,split1_test_NPV,split2_test_NPV,split3_test_NPV,split4_test_NPV,mean_test_NPV,std_test_NPV,rank_test_NPV,split0_test_PPV,split1_test_PPV,split2_test_PPV,split3_test_PPV,split4_test_PPV,mean_test_PPV,std_test_PPV,rank_test_PPV,cv,Title,RunType,MainDataset,date,classifier,FeatureSet,brt_nTrue,brt_nTop,brt_md,brt_params
0,0.090703,0.014567,0.020051,0.003527,RandomForestClassifier(),3.0,100.0,42.0,NaN,NaN,"{'clf': RandomForestClassifier(), 'clf__max_de...",0.775510,0.812500,0.770833,0.770833,0.770833,0.780102,0.016300,1,0.500000,0.550000,0.500000,0.500000,0.500000,0.510000,0.020000,33,0.590909,0.786842,0.658477,0.717445,0.710074,0.692749,0.065283,2,0.000000,0.181818,0.000000,0.000000,0.000000,0.036364,0.072727,34,0.000000,0.100000,0.000000,0.000000,0.000000,0.020000,0.040000,34,0.000000,0.100000,0.000000,0.000000,0.000000,0.020000,0.040000,34,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1,0.000000,1.000000,0.000000,0.000000,0.000000,0.200000,0.400000,34,0.775510,0.808511,0.770833,0.770833,0.770833,0.779304,0.014715,33,0.000000,1.000000,0.000000,0.000000,0.000000,0.200000,0.400000,34,5,dfcmpltPreLC_SITE5_Holdout,Holdout,dfcmpltPreLC_SITE5,<built-in method today of type object at 0x7ad...,NaN,NaN,NaN,NaN,NaN,NaN
1,0.891907,0.154319,0.081418,0.001751,RandomForestClassifier(),3.0,1000.0,42.0,NaN,NaN,"{'clf': RandomForestClassifier(), 'clf__max_de...",0.775510,0.812500,0.770833,0.770833,0.770833,0.780102,0.016300,1,0.500000,0.550000,0.500000,0.500000,0.500000,0.510000,0.020000,33,0.593301,0.784211,0.616708,0.714988,0.756757,0.693193,0.075668,1,0.000000,0.181818,0.000000,0.000000,0.000000,0.036364,0.072727,34,0.000000,0.100000,0.000000,0.000000,0.000000,0.020000,0.040000,34,0.000000,0.100000,0.000000,0.000000,0.000000,0.020000,0.040000,34,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1,0.000000,1.000000,0.000000,0.000000,0.000000,0.200000,0.400000,34,0.775510,0.808511,0.770833,0.770833,0.770833,0.779304,0.014715,33,0.000000,1.000000,0.000000,0.000000,0.000000,0.200000,0.400000,34,5,dfcmpltPreLC_SITE5_Holdout,Holdout,dfcmpltPreLC_SITE5,<built-in method today of type object at 0x7ad...,NaN,NaN,NaN,NaN,NaN,NaN
2,0.106432,0.004907,0.022866,0.000483,RandomForestClassifier(),6.0,100.0,42.0,NaN,NaN,"{'clf': RandomForestClassifier(), 'clf__max_de...",0.775510,0.812500,0.770833,0.770833,0.770833,0.780102,0.016300,1,0.500000,0.550000,0.500000,0.500000,0.500000,0.510000,0.020000,33,0.578947,0.

In [10]:
dfRes["classifier"] = dfRes["param_clf"].apply(lambda x: x.__str__().split("(")[0])
dfRes['site'] = dfRes['MainDataset'].apply(changeSite)
lColFirst = [
    "MainDataset",
    "Title",
    'site',
    "date",
    "classifier",
    'RunType',
    "cv",
    'FeatureSet',
    "brt_nTrue",
    "brt_nTop",
    'brt_md',
]
dfRes = MPutils.reorder_columns(dfRes, lColFirst)
dfRes.to_csv("OUTPUT/MP/05-classifiers/DataV3/CLFRunCombined.csv", index=False)
dfResOrg = deepcopy(dfRes)
lColsSel = lColFirst + [
    x
    for x in dfRes
    if (x.startswith("mean_test") or x.startswith("param_") or x.startswith("brt_"))
    and x not in lColFirst
]
dfRes[lColsSel].to_csv("OUTPUT/MP/05-classifiers/DataV3/CLFRunCombined_SelCol.csv")

# Go through Data and Simplify for inspection

In [11]:
dfRes = dfRes[lColsSel]

In [12]:
dfRes

,MainDataset,Title,site,date,classifier,RunType,cv,FeatureSet,brt_nTrue,brt_nTop,brt_md,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,mean_test_accuracy,mean_test_balanced_accuracy,mean_test_roc_auc,mean_test_f1,mean_test_recall,mean_test_sensitivity,mean_test_specificity,mean_test_precision,mean_test_NPV,mean_test_PPV,brt_params
0,dfcmpltPreLC_SITE5,dfcmpltPreLC_SITE5_Holdout,LUS,<built-in method today of type object at 0x7ad...,RandomForestClassifier,Holdout,5,NaN,NaN,NaN,NaN,RandomForestClassifier(),3.0,100.0,42.0,NaN,NaN,0.780102,0.510000,0.692749,0.036364,0.020000,0.020000,1.000000,0.200000,0.779304,0.200000,NaN
1,dfcmpltPreLC_SITE5,dfcmpltPreLC_SITE5_Holdout,LUS,<built-in method today of type object at 0x7ad...,RandomForestClassifier,Holdout,5,NaN,NaN,NaN,NaN,RandomForestClassifier(),3.0,1000.0,42.0,NaN,NaN,0.780102,0.510000,0.693193,0.036364,0.020000,0.020000,1.000000,0.200000,0.779304,0.200000,NaN
2,dfcmpltPreLC_SITE5,dfcmpltPreLC_SITE5_Holdout,LUS,<built-in method today of type object at 0x7ad...,RandomForestClassifier,Holdout,5,NaN,NaN,NaN,NaN,RandomForestClassifier(),6.0,100.0,42.0,NaN,NaN,0.780102,0.510000,0.652312,0.036364,0.020000,0.020000,1.000000,0.200000,0.779304,0.200000,NaN
3,dfcmpltPreLC_SITE5,dfcmpltPreLC_SITE5_Holdout,LUS,<built-in method today of type object at 0x7ad...,RandomForestClassifier,Holdout,5,NaN,NaN,NaN,NaN,RandomForestClassifier(),6.0,1000.0,42.0,NaN,NaN,0.780102,0.510000,0.674936,0.036364,0.020000,0.020000,1.000000,0.200000,0.779304,0.200000,NaN
4,dfcmpltPreLC_SITE5,dfcmpltPreLC_SITE5_Holdout,LUS,<built-in method today of type object at 0x7ad...,RandomForestClassifier,Holdout,5,NaN,NaN,NaN,NaN,RandomForestClassifier(),10.0,100.0,42.0,NaN,NaN,0.771854,0.511054,0.677128,0.064935,0.038182,0.038182,0.983926,0.266667,0.779758,0.266667,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,dfcmpltPreLC2_SITE6,dfcmpltPreLC2_SITE6_Boruta_Top,Canada,2024-05-01 21:34:03.895196,XGBClassifier,Boruta_Top,5,NaN,2.0,6.0,7.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,1.0,0.634375,0.564676,0.603757,0.392622,0.370732,0.370732,0.758621,0.419044,0.719307,0.419044,T1000_itrr500_th100_topR5
324,dfcmpltPreLC2_SITE6,dfcmpltPreLC2_SITE6_Boruta_Top,Canada,2024-05-01 21:34:03.895196,XGBClassifier,Boruta_Top,5,NaN,2.0,6.0,7.0,"XGBClassifier(base_score=None, booster=None, c...",15.0,100.0,42.0,NaN,1.0,0.639062,0.575862,0.606672,0.415253,0.400000,0.400000,0.751724,0.433285,0.726611,0.433285,T1000_itrr500_th100_topR5
325,dfcmpltPreLC2_SITE6,dfcmpltPreLC2_SITE6_Boruta_Top,Canada,2024-05-01 21:34:03.895196,XGBClassifier,Boruta_Top,5,NaN,2.0,6.0,7.0,"XGBClassifier(base_score=None, booster=None, c...",15.0,1000.0,42.0,NaN,1.0,0.634375,0.569835,0.600561,0.404921,0.390244,0.390244,0.749425,0.422719,0.723220,0.422719,T1000_itrr500_th100_topR5
326,dfcmpltPreLC2_SITE6,dfcmpltPreLC2_SITE6_Boruta_Top,Canada,2024-05-01 21:34:03.895196,XGBClassifier,Boruta_Top,5,NaN,2.0,6.0,7.0,"XGBClassifier(base_score=None, booster=None, c...",NaN,100.0,42.0,NaN,1.0,0.628125,0.562658,0.612700,0.395360,0.380488,0.380488,0.744828,0.414113,0.718490,0.414113,T1000_itrr500_th100_topR5


In [13]:
lSelMainData = [
"dfcmpltPreLC4",
"dfcmpltPreLC4_SITE1",
"dfcmpltPreLC4_SITE2",
"dfcmpltPreLC4_SITE3",
"dfcmpltPreLC4_SITE4",
"dfcmpltPreLC4_SITE5",
"dfcmpltPreLC4_SITE6",
"dfcmpltPreLC4_SITE7",
]

lPrettyCols =     [
        "MainDataset",
        "RunType",
        "classifier",
        "brt_nTrue",
        "brt_nTop",
        "accuracy",
        "balanced_accuracy",
        "roc_auc",
        "f1",
        "recall",
        "sensitivity",
        "specificity",
        "precision",
        "NPV",
        "PPV",
        "brt_md",
        'brt_params'
    ]
lPrettyColsSite = [
    "MainDataset",
    "RunType",
    'site',
    "classifier",
    "accuracy",
    "balanced_accuracy",
    "roc_auc",
    "f1",
    "recall",
    "sensitivity",
    "specificity",
    "precision",
    "NPV",
    "PPV",
]
s_brt = 'and (RunType=="Boruta_True" or RunType=="Boruta_Top")'
s_RF10 = 'classifier=="RandomForestClassifier" and param_clf__n_estimators==1000 and param_clf__max_depth==10'
s_RFXGB10 = 'param_clf__n_estimators==1000 and param_clf__max_depth==10'
s_CMPLT = 'MainDataset=="dfcmpltPreLC4"'
s_val = 'RunType=="HoldoutVal"'

In [14]:
dfRes = dfRes[dfRes['MainDataset'].isin(lSelMainData)]
dfRes.columns = [x.replace('mean_test_','') for x in dfRes.columns]

# Inspect Original Holdout
- May have duplicates

In [15]:
dfRes.query(f'{s_RF10} and RunType=="Holdout"').sort_values("balanced_accuracy", ascending=False)[lPrettyColsSite].T

,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
MainDataset,dfcmpltPreLC4_SITE4,dfcmpltPreLC4_SITE4,dfcmpltPreLC4_SITE1,dfcmpltPreLC4_SITE1,dfcmpltPreLC4_SITE3,dfcmpltPreLC4_SITE3,dfcmpltPreLC4,dfcmpltPreLC4,dfcmpltPreLC4_SITE7,dfcmpltPreLC4_SITE7,dfcmpltPreLC4_SITE6,dfcmpltPreLC4_SITE6,dfcmpltPreLC4_SITE2,dfcmpltPreLC4_SITE2,dfcmpltPreLC4_SITE5,dfcmpltPreLC4_SITE5
RunType,Holdout,Holdout,Holdout,Holdout,Holdout,Holdout,Holdout,Holdout,Holdout,Holdout,Holdout,Holdout,Holdout,Holdout,Holdout,Holdout
site,RIO,RIO,LDN,LDN,SAN,SAN,Global,Global,NorthAmerica,NorthAmerica,Canada,Canada,MTL,MTL,LUS,LUS
classifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier
accuracy,1.0,1.0,0.912121,0.912121,0.865934,0.865934,0.71582,0.71582,0.721814,0.721814,0.708149,0.708149,0.694963,0.694963,0.77602,0.77602
balanced_accuracy,1.0,1.0,0.912143,0.912143,0.826061,0.826061,0.658054,0.658054,0.625205,0.625205,0.556341,0.556341,0.52096,0.52096,0.507368,0.507368
roc_auc,1.0,1.0,0.952857,0.952857,0.904242,0.904242,0.773869,0.773869,0.769748,0.769748,0.723729,0.723729,0.676959,0.676959,0.673141,0.673141
f1,1.0,1.0,0.892929,0.892929,0.911104,0.911104,0.511843,0.511843,0.414798,0.414798,0.238184,0.238184,0.118133,0.118133,0.036364,0.036364
recall,1.0,1.0,0.91,0.91,0.885455,0.885455,0.375131,0.375131,0.277176,0.277176,0.145,0.145,0.066984,0.066984,0.02,0.02
sensitivity,1.0,1.0,0.91,0.91,0.885455,0.885455,0.375131,0.375131,0.277176,0.277176,0.145,0.145,0.066984,0.066984,0.02,0.02


# Per site - Best Results

In [16]:
dfRes.query(f'{s_RF10} and RunType=="Boruta_True"').sort_values('balanced_accuracy',ascending=False)

,MainDataset,Title,site,date,classifier,RunType,cv,FeatureSet,brt_nTrue,brt_nTop,brt_md,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_params
87,dfcmpltPreLC4_SITE4,dfcmpltPreLC4_SITE4_Boruta_True,RIO,2024-05-06 10:48:45.301840,RandomForestClassifier,Boruta_True,5,NaN,19.0,23.0,3.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.981048,0.946667,0.998462,0.988819,1.000000,1.000000,0.893333,0.978307,1.000000,0.978307,T1000_itrr500_th100_topR5
251,dfcmpltPreLC4_SITE4,dfcmpltPreLC4_SITE4_Boruta_True,RIO,2024-05-06 10:48:45.301840,RandomForestClassifier,Boruta_True,5,NaN,11.0,16.0,7.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.981048,0.946667,0.996923,0.988819,1.000000,1.000000,0.893333,0.978307,1.000000,0.978307,T1000_itrr500_th100_topR5
169,dfcmpltPreLC4_SITE4,dfcmpltPreLC4_SITE4_Boruta_True,RIO,2024-05-06 10:48:45.301840,RandomForestClassifier,Boruta_True,5,NaN,11.0,15.0,5.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.981048,0.946667,0.996923,0.988819,1.000000,1.000000,0.893333,0.978307,1.000000,0.978307,T1000_itrr500_th100_topR5
251,dfcmpltPreLC4_SITE3,dfcmpltPreLC4_SITE3_Boruta_True,SAN,2024-05-06 10:41:21.307882,RandomForestClassifier,Boruta_True,5,NaN,3.0,7.0,7.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.926374,0.929394,0.956061,0.950426,0.925455,0.925455,0.933333,0.981818,0.820000,0.981818,T1000_itrr500_th100_topR5
169,dfcmpltPreLC4_SITE3,dfcmpltPreLC4_SITE3_Boruta_True,SAN,2024-05-06 10:41:21.307882,RandomForestClassifier,Boruta_True,5,NaN,5.0,9.0,5.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.910989,0.855152,0.963030,0.942733,0.943636,0.943636,0.766667,0.943333,0.833333,0.943333,T1000_itrr500_th100_topR5
87,dfcmpltPreLC4_SITE3,dfcmpltPreLC4_SITE3_Boruta_True,SAN,2024-05-06 10:41:21.307882,RandomForestClassifier,Boruta_True,5,NaN,13.0,17.0,3.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.910989,0.855152,0.950909,0.942733,0.943636,0.943636,0.766667,0.943333,0.833333,0.943333,T1000_itrr500_th100_topR5
87,dfcmpltPreLC4_SITE1,dfcmpltPreLC4_SITE1_Boruta_True,LDN,2024-05-06 10:20:07.740764,RandomForestClassifier,Boruta_True,5,NaN,15.0,19.0,3.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.804545,0.788571,0.880000,0.743492,0.720000,0.720000,0.857143,0.793333,0.831429,0.793333,T1000_itrr500_th100_topR5
169,dfcmpltPreLC4_SITE1,dfcmpltPreLC4_SITE1_Boruta_True,LDN,2024-05-06 10:20:07.740764,RandomForestClassifier,Boruta_True,5,NaN,6.0,10.0,5.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.768182,0.770714,0.881429,0.736580,0.770000,0.770000,0.771429,0.761667,0.830556,0.761667,T1000_itrr500_th100_topR5
251,dfcmpltPreLC4_SITE1,dfcmpltPreLC4_SITE1_Boruta_True,LDN,2024-05-06 10:20:07.740764,RandomForestClassifier,Boruta_True,5,NaN,6.0,10.0,7.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.768182,0.770714,0.881429,0.736580,0.770000,0.770000,0.771429,0.761667,0.830556,0.761667,T1000_itrr500_th100_topR5
87,dfcmpltPreLC4,dfcmpltPreLC4_Boruta_True,Global,2024-05-06 10:08:36.667108,RandomForestClassifier,Boruta_True,5,NaN,21.0,26.0,3.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.688497,0.638307,0.734955,0.501326,0.394488,0.394488,0.882126,0.691237,0.688649,0.691237,T1000_itrr500_th100_topR5


In [27]:
# dfTmp = dfRes.query(f'{s_RF10} and RunType=="Boruta_True"').sort_values('balanced_accuracy',ascending=False)
dfTmp = dfRes.query(f'RunType=="Boruta_True" and {s_RFXGB10}').sort_values('balanced_accuracy',ascending=False)

lTopBrt = []
for i in lSelMainData:
    dfTmpTop = dfTmp.query(f'MainDataset=="{i}"').sort_values('balanced_accuracy',ascending=False)
    lTopBrt.append(dfTmpTop.iloc[0,:])

In [28]:
pd.DataFrame(lTopBrt)[lPrettyColsSite+['brt_md','brt_nTrue','Title']].sort_values('balanced_accuracy',ascending=False)

,MainDataset,RunType,site,classifier,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_md,brt_nTrue,Title
108,dfcmpltPreLC4_SITE4,Boruta_True,RIO,XGBClassifier,0.987298,0.963333,0.997436,0.992453,1.000000,1.000000,0.926667,0.985185,1.000000,0.985185,3.0,19.0,dfcmpltPreLC4_SITE4_Boruta_True
251,dfcmpltPreLC4_SITE3,Boruta_True,SAN,RandomForestClassifier,0.926374,0.929394,0.956061,0.950426,0.925455,0.925455,0.933333,0.981818,0.820000,0.981818,7.0,3.0,dfcmpltPreLC4_SITE3_Boruta_True
180,dfcmpltPreLC4_SITE1,Boruta_True,LDN,XGBClassifier,0.822727,0.824286,0.898571,0.786580,0.820000,0.820000,0.828571,0.816667,0.901984,0.816667,5.0,6.0,dfcmpltPreLC4_SITE1_Boruta_True
98,dfcmpltPreLC4_SITE7,Boruta_True,NorthAmerica,XGBClassifier,0.670436,0.639128,0.665584,0.535374,0.527843,0.527843,0.750412,0.545507,0.738903,0.545507,3.0,18.0,dfcmpltPreLC4_SITE7_Boruta_True
87,dfcmpltPreLC4,Boruta_True,Global,RandomForestClassifier,0.688497,0.638307,0.734955,0.501326,0.394488,0.394488,0.882126,0.691237,0.688649,0.691237,3.0,21.0,dfcmpltPreLC4_Boruta_True
108,dfcmpltPreLC4_SITE2,Boruta_True,MTL,XGBClassifier,0.672489,0.607897,0.613859,0.453217,0.440794,0.440794,0.775000,0.468233,0.757557,0.468233,3.0,10.0,dfcmpltPreLC4_SITE2_Boruta_True
118,dfcmpltPreLC4_SITE6,Boruta_True,Canada,XGBClassifier,0.656205,0.598358,0.617495,0.445285,0.442692,0.442692,0.754023,0.450340,0.747817,0.450340,3.0,14.0,dfcmpltPreLC4_SITE6_Boruta_True
169,dfcmpltPreLC4_SITE5,Boruta_True,LUS,RandomForestClassifier,0.717602,0.488213,0.419454,0.102991,0.072727,0.072727,0.903698,0.208571,0.771361,0.208571,5.0,2.0,dfcmpltPreLC4_SITE5_Boruta_True


In [21]:
pd.DataFrame(lTopBrt)[['brt_md','brt_nTrue','Title']+lPrettyColsSite].sort_values('MainDataset',ascending=True)

,brt_md,brt_nTrue,Title,MainDataset,RunType,site,classifier,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV
87,3.0,21.0,dfcmpltPreLC4_Boruta_True,dfcmpltPreLC4,Boruta_True,Global,RandomForestClassifier,0.688497,0.638307,0.734955,0.501326,0.394488,0.394488,0.882126,0.691237,0.688649,0.691237
87,3.0,15.0,dfcmpltPreLC4_SITE1_Boruta_True,dfcmpltPreLC4_SITE1,Boruta_True,LDN,RandomForestClassifier,0.804545,0.788571,0.880000,0.743492,0.720000,0.720000,0.857143,0.793333,0.831429,0.793333
87,3.0,10.0,dfcmpltPreLC4_SITE2_Boruta_True,dfcmpltPreLC4_SITE2,Boruta_True,MTL,RandomForestClassifier,0.688096,0.592381,0.666842,0.400498,0.344762,0.344762,0.840000,0.480362,0.744137,0.480362
251,7.0,3.0,dfcmpltPreLC4_SITE3_Boruta_True,dfcmpltPreLC4_SITE3,Boruta_True,SAN,RandomForestClassifier,0.926374,0.929394,0.956061,0.950426,0.925455,0.925455,0.933333,0.981818,0.820000,0.981818
87,3.0,19.0,dfcmpltPreLC4_SITE4_Boruta_True,dfcmpltPreLC4_SITE4,Boruta_True,RIO,RandomForestClassifier,0.981048,0.946667,0.998462,0.988819,1.000000,1.000000,0.893333,0.978307,1.000000,0.978307
169,5.0,2.0,dfcmpltPreLC4_SITE5_Boruta_True,dfcmpltPreLC4_SITE5,Boruta_True,LUS,RandomForestClassifier,0.717602,0.488213,0.419454,0.102991,0.072727,0.072727,0.903698,0.208571,0.771361,0.208571
87,3.0,14.0,dfcmpltPreLC4_SITE6_Boruta_True,dfcmpltPreLC4_SITE6,Boruta_True,Canada,RandomForestClassifier,0.698725,0.597880,0.687070,0.406034,0.326795,0.326795,0.868966,0.540606,0.738110,0.540606
87,3.0,18.0,dfcmpltPreLC4_SITE7_Boruta_True,dfcmpltPreLC4_SITE7,Boruta_True,NorthAmerica,RandomForestClassifier,0.697538,0.631497,0.751774,0.485046,0.396627,0.396627,0.866367,0.626783,0.719036,0.626783


In [22]:
# # Create the ExcelWriter object
writer = pd.ExcelWriter('../OUTPUTS/prelim_preLongCOVID4_Brt_columns.xlsx', engine='xlsxwriter')
pd.DataFrame(lTopBrt)[lPrettyColsSite+['brt_md','brt_nTrue','Title']].sort_values('balanced_accuracy',ascending=False).to_excel(writer,sheet_name='Results')

for i,d in pd.DataFrame(lTopBrt).iterrows():
    tmpdataset = d['MainDataset']
    md = int(d['brt_md'])
    f = f'OUTPUT/MP/05-classifiers/DataV3/{tmpdataset}/FS_{tmpdataset}_Boruta_T1000_itrr500_th100_topR5_MD{md}.csv'
    dfeat = pd.read_csv(f)
    MPutils.getTrueFeatList(f,asDF=True).to_excel(writer, sheet_name=d['site'])
    


    # Write each dataframe to a different sheet
    # df1.to_excel(writer, sheet_name=d['site'])

#     # Save the Excel file
writer.close()


In [23]:
dfTmpTop

,MainDataset,Title,site,date,classifier,RunType,cv,FeatureSet,brt_nTrue,brt_nTop,brt_md,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_params
87,dfcmpltPreLC4_SITE7,dfcmpltPreLC4_SITE7_Boruta_True,NorthAmerica,2024-05-06 11:20:32.414037,RandomForestClassifier,Boruta_True,5,NaN,18.0,23.0,3.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.697538,0.631497,0.751774,0.485046,0.396627,0.396627,0.866367,0.626783,0.719036,0.626783,T1000_itrr500_th100_topR5
169,dfcmpltPreLC4_SITE7,dfcmpltPreLC4_SITE7_Boruta_True,NorthAmerica,2024-05-06 11:20:32.414037,RandomForestClassifier,Boruta_True,5,NaN,9.0,13.0,5.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.674691,0.620732,0.690659,0.487352,0.428706,0.428706,0.812759,0.566064,0.716693,0.566064,T1000_itrr500_th100_topR5
251,dfcmpltPreLC4_SITE7,dfcmpltPreLC4_SITE7_Boruta_True,NorthAmerica,2024-05-06 11:20:32.414037,RandomForestClassifier,Boruta_True,5,NaN,3.0,8.0,7.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.666221,0.615895,0.686167,0.483196,0.436784,0.436784,0.795006,0.548038,0.715991,0.548038,T1000_itrr500_th100_topR5


In [26]:
s_query = f'{s_CMPLT} and param_clf__n_estimators==1000 and param_clf__max_depth==10 {s_brt}'
dfTmp = deepcopy(dfRes.query(s_query)).sort_values('balanced_accuracy',ascending=False)
print(s_query)
print(f"Max balanced accuracy {dfTmp['balanced_accuracy'].max()}")
# print(f"Max balanced accuracy {dfTmp['balanced_accuracy'].max()}")

dfTmp[lPrettyCols]

MainDataset=="dfcmpltPreLC4" and param_clf__n_estimators==1000 and param_clf__max_depth==10 and (RunType=="Boruta_True" or RunType=="Boruta_Top")
Max balanced accuracy 0.6383068761244804


,MainDataset,RunType,classifier,brt_nTrue,brt_nTop,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_md,brt_params
87,dfcmpltPreLC4,Boruta_True,RandomForestClassifier,21.0,26.0,0.688497,0.638307,0.734955,0.501326,0.394488,0.394488,0.882126,0.691237,0.688649,0.691237,3.0,T1000_itrr500_th100_topR5
128,dfcmpltPreLC4,Boruta_Top,RandomForestClassifier,21.0,26.0,0.686687,0.633692,0.742070,0.487581,0.376202,0.376202,0.891183,0.700128,0.684530,0.700128,3.0,T1000_itrr500_th100_topR5
210,dfcmpltPreLC4,Boruta_Top,RandomForestClassifier,11.0,15.0,0.661179,0.624555,0.704786,0.510488,0.447048,0.447048,0.802062,0.597700,0.688295,0.597700,5.0,T1000_itrr500_th100_topR5
169,dfcmpltPreLC4,Boruta_True,RandomForestClassifier,11.0,15.0,0.652080,0.622142,0.692515,0.520314,0.476933,0.476933,0.767350,0.573788,0.690644,0.573788,5.0,T1000_itrr500_th100_topR5
292,dfcmpltPreLC4,Boruta_Top,RandomForestClassifier,3.0,7.0,0.638423,0.612040,0.673701,0.514927,0.483934,0.483934,0.740146,0.550738,0.685455,0.550738,7.0,T1000_itrr500_th100_topR5
190,dfcmpltPreLC4,Boruta_True,XGBClassifier,11.0,15.0,0.624745,0.606599,0.640633,0.521692,0.518234,0.518234,0.694965,0.527421,0.687428,0.527421,5.0,T1000_itrr500_th100_topR5
139,dfcmpltPreLC4,Boruta_Top,XGBClassifier,21.0,26.0,0.628377,0.604509,0.670889,0.511344,0.488558,0.488558,0.720460,0.538558,0.680886,0.538558,3.0,T1000_itrr500_th100_topR5
98,dfcmpltPreLC4,Boruta_True,XGBClassifier,21.0,26.0,0.622017,0.600768,0.662748,0.511561,0.497675,0.497675,0.703862,0.528477,0.679819,0.528477,3.0,T1000_itrr500_th100_topR5
149,dfcmpltPreLC4,Boruta_Top,XGBClassifier,21.0,26.0,0.622022,0.598428,0.666641,0.504531,0.483960,0.483960,0.712896,0.529610,0.676780,0.529610,3.0,T1000_itrr500_th100_topR5
180,dfcmpltPreLC4,Boruta_True,XGBClassifier,11.0,15.0,0.617447,0.598176,0.643869,0.509245,0.504441,0.504441,0.691912,0.516846,0.680938,0.516846,5.0,T1000_itrr500_th100_topR5


# Difference between Boruta Selected Datasets

In [21]:
lbrt_7 = MPutils.getTrueFeatList(
    "OUTPUT/MP/05-classifiers/DataV3/dfcmplt/FS_dfcmplt_Boruta_T1000_itrr500_th100_topR5_MD7.csv"
)

lbrt_5 = MPutils.getTrueFeatList(
    "OUTPUT/MP/05-classifiers/DataV3/dfcmplt/FS_dfcmplt_Boruta_T1000_itrr500_th100_topR5_MD5.csv"
)

lbrt_3 = MPutils.getTrueFeatList(
    "OUTPUT/MP/05-classifiers/DataV3/dfcmplt/FS_dfcmplt_Boruta_T1000_itrr500_th100_topR5_MD3.csv"
)

len(lbrt_7), len(lbrt_5), len(lbrt_3)

(68, 81, 113)

In [24]:
set(lbrt_7).difference(set(lbrt_5))

set()

In [27]:
set(lbrt_5).difference(set(lbrt_7))

{'CD-covid_test_result_2',
 'CD-gcs',
 'CD-neurologic-other-1',
 'CD-neurologic-other-4',
 'CD-required_oxygen',
 'CD-respiratory_rate',
 'CD-severity_level___2',
 'LABS-IND-basophil_count',
 'LABS-IND-eosinophil',
 'LABS-IND-pct',
 'SYMPT-fever_38_0oc___2',
 'SYMPT-night_sweats___2',
 'SYMPT-runny_nose_rhinorrhea___2'}

In [32]:
set(lbrt_7).difference(set(lbrt_3))

set()

In [33]:
set(lbrt_3).difference(set(lbrt_5))

{'CD-covid_test_result_1',
 'CD-covid_test_result_3',
 'CD-hospitalized',
 'CD-insomnia___1',
 'CD-medication_received___4',
 'CD-medication_received___5',
 'CD-memory_problems___1',
 'CD-respiratory-other-1',
 'CD-severity_level___3',
 'CD-type_of_care___2',
 'HHX-Arterial Hypertension',
 'HHX-Diabetes',
 'HHX-OTHER-Sleep-disorder',
 'HHX-Obesity',
 'IMG-ctt',
 'LAB-test-lung_function',
 'LABS-IND-cpk',
 'LABS-IND-inr',
 'MED-Cardiovascular-agents',
 'MED-Respiratory-tract-agents',
 'SYMPT-cough___1',
 'SYMPT-diarrhea___1',
 'SYMPT-extremity_weakness_or_numb___1',
 'SYMPT-fatigue___1',
 'SYMPT-fever_38_0oc___1',
 'SYMPT-joint_pain_arthralgia___1',
 'SYMPT-muscle_aches_myalgia___1',
 'SYMPT-nausea_vomiting___1',
 'SYMPT-runny_nose_rhinorrhea___1',
 'SYMPT-sore_throat___1',
 'SYMPT-trouble_speaking_aphasia_d___2',
 'SYMPT-weight_loss___1'}

In [34]:
set(lbrt_3).difference(set(lbrt_5))

{'CD-covid_test_result_1',
 'CD-covid_test_result_3',
 'CD-hospitalized',
 'CD-insomnia___1',
 'CD-medication_received___4',
 'CD-medication_received___5',
 'CD-memory_problems___1',
 'CD-respiratory-other-1',
 'CD-severity_level___3',
 'CD-type_of_care___2',
 'HHX-Arterial Hypertension',
 'HHX-Diabetes',
 'HHX-OTHER-Sleep-disorder',
 'HHX-Obesity',
 'IMG-ctt',
 'LAB-test-lung_function',
 'LABS-IND-cpk',
 'LABS-IND-inr',
 'MED-Cardiovascular-agents',
 'MED-Respiratory-tract-agents',
 'SYMPT-cough___1',
 'SYMPT-diarrhea___1',
 'SYMPT-extremity_weakness_or_numb___1',
 'SYMPT-fatigue___1',
 'SYMPT-fever_38_0oc___1',
 'SYMPT-joint_pain_arthralgia___1',
 'SYMPT-muscle_aches_myalgia___1',
 'SYMPT-nausea_vomiting___1',
 'SYMPT-runny_nose_rhinorrhea___1',
 'SYMPT-sore_throat___1',
 'SYMPT-trouble_speaking_aphasia_d___2',
 'SYMPT-weight_loss___1'}

In [35]:
set(lbrt_5).difference(set(lbrt_3))

set()

In [30]:
set(lbrt_5).difference(set(lbrt_7))

{'CD-covid_test_result_2',
 'CD-gcs',
 'CD-neurologic-other-1',
 'CD-neurologic-other-4',
 'CD-required_oxygen',
 'CD-respiratory_rate',
 'CD-severity_level___2',
 'LABS-IND-basophil_count',
 'LABS-IND-eosinophil',
 'LABS-IND-pct',
 'SYMPT-fever_38_0oc___2',
 'SYMPT-night_sweats___2',
 'SYMPT-runny_nose_rhinorrhea___2'}

# Validation Results

In [71]:
dfVal = dfRes.query(f'{s_RF10} and {s_val}').sort_values('balanced_accuracy',ascending=False)
dfVal

,MainDataset,Title,site,date,classifier,RunType,cv,FeatureSet,brt_nTrue,brt_nTop,brt_md,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_params
5,dfcmpltPreLC4_SITE4,dfcmpltPreLC4_SITE4_HoldoutVal,RIO,2024-05-06 11:44:10.958993,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC4_SITE4_Boruta_T1000_itrr500_th...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,NaN
5,dfcmpltPreLC4_SITE4,dfcmpltPreLC4_SITE4_HoldoutVal,RIO,2024-05-06 11:44:08.270017,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC4_Boruta_T1000_itrr500_th100_to...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.980645,0.940000,0.979231,0.988679,1.000000,1.000000,0.880000,0.977778,1.000000,0.977778,NaN
5,dfcmpltPreLC4_SITE1,dfcmpltPreLC4_SITE1_HoldoutVal,LDN,2024-05-06 11:43:52.601321,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC4_Boruta_T1000_itrr500_th100_to...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.877273,0.878571,0.930000,0.858608,0.900000,0.900000,0.857143,0.845000,0.941667,0.845000,NaN
5,dfcmpltPreLC4_SITE3,dfcmpltPreLC4_SITE3_HoldoutVal,SAN,2024-05-06 11:44:06.063062,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC4_SITE3_Boruta_T1000_itrr500_th...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.852747,0.832727,0.898182,0.901010,0.865455,0.865455,0.800000,0.945455,0.640000,0.945455,NaN
5,dfcmpltPreLC4_SITE1,dfcmpltPreLC4_SITE1_HoldoutVal,LDN,2024-05-06 11:43:54.701532,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC4_SITE1_Boruta_T1000_itrr500_th...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.827273,0.829286,0.940000,0.779243,0.830000,0.830000,0.828571,0.805000,0.906667,0.805000,NaN
5,dfcmpltPreLC4_SITE3,dfcmpltPreLC4_SITE3_HoldoutVal,SAN,2024-05-06 11:44:04.053039,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC4_Boruta_T1000_itrr500_th100_to...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.850549,0.767576,0.895152,0.901485,0.901818,0.901818,0.633333,0.909382,0.763333,0.909382,NaN
5,dfcmpltPreLC4_SITE7,dfcmpltPreLC4_SITE7_HoldoutVal,NorthAmerica,2024-05-06 11:44:29.614930,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC4_SITE7_Boruta_T1000_itrr500_th...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.744620,0.684352,0.771235,0.564820,0.466980,0.466980,0.901723,0.719913,0.750855,0.719913,NaN
5,dfcmpltPreLC4,dfcmpltPreLC4_HoldoutVal,Global,2024-05-06 11:43:49.648575,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC4_Boruta_T1000_itrr500_th100_to...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.711287,0.679182,0.763237,0.588057,0.521682,0.521682,0.836683,0.682757,0.726625,0.682757,NaN
5,dfcmpltPreLC4_SITE7,dfcmpltPreLC4_SITE7_HoldoutVal,NorthAmerica,2024-05-06 11:44:25.865340,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC4_Boruta_T1000_itrr500_th100_to...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.693212,0.623336,0.728322,0.466846,0.371765,0.371765,0.874906,0.633154,0.711257,0.633154,NaN
5,dfcmpltPreLC4_SITE6,dfcmpltPreLC4_SITE6_HoldoutVal,Canada,2024-05-06 11:44:22.804672,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC4_SITE6_Boruta_T1000_itrr500_th...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.689264,0.581675,0.673098,0.367057,0.290000,0.290000,0.873349,0.518000,0.727887,0.518000,NaN


In [72]:
dfVal = dfVal.sort_values(['FeatureSet','MainDataset'])
dfVal

,MainDataset,Title,site,date,classifier,RunType,cv,FeatureSet,brt_nTrue,brt_nTop,brt_md,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_params
5,dfcmpltPreLC4,dfcmpltPreLC4_HoldoutVal,Global,2024-05-06 11:43:49.648575,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC4_Boruta_T1000_itrr500_th100_to...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.711287,0.679182,0.763237,0.588057,0.521682,0.521682,0.836683,0.682757,0.726625,0.682757,NaN
5,dfcmpltPreLC4_SITE1,dfcmpltPreLC4_SITE1_HoldoutVal,LDN,2024-05-06 11:43:52.601321,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC4_Boruta_T1000_itrr500_th100_to...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.877273,0.878571,0.930000,0.858608,0.900000,0.900000,0.857143,0.845000,0.941667,0.845000,NaN
5,dfcmpltPreLC4_SITE2,dfcmpltPreLC4_SITE2_HoldoutVal,MTL,2024-05-06 11:43:58.213916,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC4_Boruta_T1000_itrr500_th100_to...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.670630,0.545394,0.662683,0.291222,0.218730,0.218730,0.872057,0.448111,0.714268,0.448111,NaN
5,dfcmpltPreLC4_SITE3,dfcmpltPreLC4_SITE3_HoldoutVal,SAN,2024-05-06 11:44:04.053039,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC4_Boruta_T1000_itrr500_th100_to...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.850549,0.767576,0.895152,0.901485,0.901818,0.901818,0.633333,0.909382,0.763333,0.909382,NaN
5,dfcmpltPreLC4_SITE4,dfcmpltPreLC4_SITE4_HoldoutVal,RIO,2024-05-06 11:44:08.270017,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC4_Boruta_T1000_itrr500_th100_to...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.980645,0.940000,0.979231,0.988679,1.000000,1.000000,0.880000,0.977778,1.000000,0.977778,NaN
5,dfcmpltPreLC4_SITE5,dfcmpltPreLC4_SITE5_HoldoutVal,LUS,2024-05-06 11:44:13.912094,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC4_Boruta_T1000_itrr500_th100_to...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.746854,0.514073,0.600067,0.134380,0.092727,0.092727,0.935420,0.363810,0.781001,0.363810,NaN
5,dfcmpltPreLC4_SITE6,dfcmpltPreLC4_SITE6_HoldoutVal,Canada,2024-05-06 11:44:19.846220,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC4_Boruta_T1000_itrr500_th100_to...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.686077,0.577972,0.710466,0.364025,0.285000,0.285000,0.870944,0.504324,0.725522,0.504324,NaN
5,dfcmpltPreLC4_SITE7,dfcmpltPreLC4_SITE7_HoldoutVal,NorthAmerica,2024-05-06 11:44:25.865340,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC4_Boruta_T1000_itrr500_th100_to...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.693212,0.623336,0.728322,0.466846,0.371765,0.371765,0.874906,0.633154,0.711257,0.633154,NaN
5,dfcmpltPreLC4_SITE1,dfcmpltPreLC4_SITE1_HoldoutVal,LDN,2024-05-06 11:43:54.701532,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC4_SITE1_Boruta_T1000_itrr500_th...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.827273,0.829286,0.940000,0.779243,0.830000,0.830000,0.828571,0.805000,0.906667,0.805000,NaN
5,dfcmpltPreLC4_SITE2,dfcmpltPreLC4_SITE2_HoldoutVal,MTL,2024-05-06 11:44:01.682305,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC4_SITE2_Boruta_T1000_itrr500_th...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.665592,0.555536,0.606742,0.327077,0.269048,0.269048,0.842025,0.437266,0.721369,0.437266,NaN


In [74]:
sGlobal = 'FS_dfcmpltPreLC4_Boruta_T1000_itrr500_th100_topR5_MD3'

dfVal['FeatureSetSpecific'] = dfVal['FeatureSet'].apply(lambda x:  'Global' if x == sGlobal else 'Site')

In [75]:
# # Create the ExcelWriter object
writer = pd.ExcelWriter('../OUTPUTS/prelim_preLongCOVID4_Val.xlsx', engine='xlsxwriter')
dfVal[lPrettyColsSite+['brt_md','brt_nTrue','Title','FeatureSet','FeatureSetSpecific']].sort_values('balanced_accuracy',ascending=False).to_excel(writer,sheet_name='Results')


for i,d in pd.DataFrame(dfVal.sort_values([])).iterrows():
    tmpFS = d['FeatureSet']
    sheetName = f"{d['FeatureSetSpecific']} Feat - {d['site']}"
        
    tmpMainData = d['MainDataset']
    
    dFeat = pd.read_csv(f'OUTPUT/MP/05-classifiers/DataV3/{tmpMainData}/FR_{tmpMainData}_HoldoutVal_{tmpFS}.csv')
    # f = f'OUTPUT/MP/05-classifiers/DataV3/{tmpdataset}/FS_{tmpdataset}_Boruta_T1000_itrr500_th100_topR5_MD{md}.csv'
    # dfeat = pd.read_csv(f)
    
    # MPutils.getTrueFeatList(f,asDF=True).to_excel(writer, sheet_name=d['site'])
    dFeat.to_excel(writer,sheet_name=sheetName)


    # Write each dataframe to a different sheet
    # df1.to_excel(writer, sheet_name=d['site'])

#     # Save the Excel file
writer.close()
